Install required packages

In [ ]:
pip install bs4 && pip install requests && pip install texttable

Import required packages

In [ ]:
import requests
from bs4 import BeautifulSoup as BS
import csv

Define some functions

In [ ]:
def get_content(url):
  return requests.get(url).content

def create_bs(url):
  return BS(get_content(url), 'html5lib')

Scrape Amazon India website to extract information about various models of mobile phone manufactured by Redmi and samsung. (Model Name, price, ratings). Save it in a CSV file.

In [ ]:
def extract_rating(rating):
  if 'Stars & Up' in rating or 'Star & Up' in rating:
    rating = float(rating[0])
  else:
    rating = float(rating[:3])
  return rating

In [ ]:
bs = create_bs('https://www.amazon.in/s?i=electronics&bbn=1389401031&rh=n%3A1389401031%2Cp_89%3ARedmi%7CSamsung&dc&qid=1611369268&rnid=3837712031&ref=sr_nr_p_89_3')
prices = bs.findAll('span', {'class': 'a-price-whole'})
names = bs.findAll('span', {'class': 'a-text-normal'})
ratings = bs.findAll('span', {'class': 'a-icon-alt'})

file_name = 'phones.csv'
with open(file_name, 'w') as file:
  writer = csv.DictWriter(file, ['Name', 'Price', 'Rating'])
  writer.writeheader()
  for idx, price in enumerate(prices):
    price = float(price.text.replace(',',''))
    name = names[idx].text
    rating = extract_rating(ratings[idx].text)
    writer.writerow({'Name': name,
                     'Price': price,
                     'Rating': rating})

# EDA

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('phones.csv')
df

,Name,Price,Rating
0,"Redmi 9 Power (Blazing Blue, 4GB RAM, 64GB Sto...",10999.0,4.0
1,"Samsung Galaxy M31 Prime Edition (Space Black,...",15999.0,3.0
2,Redmi 9 Power (Mighty Black 4GB RAM 64GB Stora...,10999.0,2.0
3,"Redmi 9 Power (Electric Green, 4GB RAM, 64GB S...",10999.0,1.0
4,"Redmi 9A (Sea Blue, 2GB Ram, 32GB Storage)",6999.0,3.7
5,"Redmi Note 9 Pro (Aurora Blue, 4GB RAM, 64GB S...",12999.0,4.2
6,"Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB...",15999.0,3.7
7,"Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64...",12499.0,3.7
8,"Redmi 9 Power (Fiery Red, 4GB RAM, 64GB Storag...",10999.0,4.1
9,"Redmi Note 9 Pro Max (Aurora Blue, 6GB RAM, 64...",14999.0,4.2


1. Number of mobileset in different price band - (1 - 5K, 5K - 10k, 10K and above) 

In [ ]:
def name_contains(df, token):
  return df[df['Name'].str.contains(token)]

brands = ['Samsung', 'Redmi']
brand_phones = [name_contains(df, brand) for brand in brands]

In [ ]:
price_bands = ['Price >= 1000 & Price < 5000',
               'Price >= 5000 & Price < 10000',
               'Price > 10000']

In [ ]:
def draw_table(data, header):
  tbl = tt.Texttable()
  tbl.add_rows([header] + data);
  print(tbl.draw())

within manufacturer

In [ ]:
import texttable as tt
data = []
for brand, bdf in zip(brands, brand_phones):
  for band in price_bands:
    data.append((brand, band, bdf.query(band).shape[0]))

draw_table(data, ('Brand', 'Price Band', '# of Sets'))

+---------+-------------------------------+-----------+
|  Brand  |          Price Band           | # of Sets |
+=========+===============================+===========+
| Samsung | Price >= 1000 & Price < 5000  | 0         |
+---------+-------------------------------+-----------+
| Samsung | Price >= 5000 & Price < 10000 | 1         |
+---------+-------------------------------+-----------+
| Samsung | Price > 10000                 | 8         |
+---------+-------------------------------+-----------+
| Redmi   | Price >= 1000 & Price < 5000  | 0         |
+---------+-------------------------------+-----------+
| Redmi   | Price >= 5000 & Price < 10000 | 5         |
+---------+-------------------------------+-----------+
| Redmi   | Price > 10000                 | 10        |
+---------+-------------------------------+-----------+


between manufacturer.

In [ ]:
data = []
for band in price_bands:
  data.append((band, df.query(band).shape[0]))

draw_table(data, ('Price Band', '# of Sets'))

+-------------------------------+-----------+
|          Price Band           | # of Sets |
+===============================+===========+
| Price >= 1000 & Price < 5000  | 0         |
+-------------------------------+-----------+
| Price >= 5000 & Price < 10000 | 6         |
+-------------------------------+-----------+
| Price > 10000                 | 18        |
+-------------------------------+-----------+


2. Highest rated Mobile set in each manufacturer.

In [ ]:
for brand, bdf in zip(brands, brand_phones):               
  print(f'\nBrand: {brand}')
  print(bdf[bdf['Rating'] == bdf['Rating'].max()])


Brand: Samsung
                                                 Name    Price  Rating
17   Samsung Galaxy M02s (Blue,4GB RAM, 64GB Storage)   9999.0     4.3
23  Samsung Galaxy M51 (Celestial Black, 6GB RAM, ...  22999.0     4.3

Brand: Redmi
                                                 Name    Price  Rating
21  Redmi Note 9 Pro Max (Aurora Blue, 6GB RAM, 12...  17499.0     4.4


In [ ]:
pip install sweetviz

In [ ]:
import sweetviz as sv

my_report = sv.analyze(df)
my_report.show_html()

                                             |          | [  0%]   00:00 -> (? left)

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
